# JupyterHubのセットアップ

---

VCノードにJupyterHub環境を構築します。

具体的には、Jupyterhubシステムの構成要素である、各Dockerコンテナを起動します。  
また、single-user notebook serverコンテナイメージの取得・もしくは作成を行います。

## 概要

このNotebookで構築するJupyterHubの構成要素を以下に示します。

![<構成図表示エラー>](images/121/arch-121.png)

> このNotebookで構築するJupyterHubではMoodleとの連携を前提としています。

### 事前に準備が必要となるものについて

このNotebookを実行するにあたって、あらかじめ準備が必要となるものについて以下に記します。

* VCノード構築
* JupyterHubのサーバ証明書
    - JupyterHubではHTTPSによる公開を行うためサーバ証明書とその秘密鍵を準備する必要があります。
* 機関のLDAPサーバ
* Moodleシステム
    - 本ノートブックで構築するJupyterhubと、lti1.3を用いた認証連携が可能な外部ツール設定を完了していること。（外部ツール設定時に自動発行されるキーを本ノートブック上で指定するため）
        - 未設定の場合は、先に設定を行ってください。
        - 設定例として、ノートブック一覧の「021-Moodleの外部ツール設定例（lti1.3&JupyterHub）.ipynb」を参考にしてください。

### UnitGroup名

JupyterHubの構築環境となるVCPのUnitGroup名を指定します。

VCノードを作成時に指定した値を確認するために `group_vars`ファイル名の一覧を表示します。

In [ ]:
!ls -1 group_vars/

上のセルの出力結果を参考にして、UnitGroup名を次のセルに指定してください。

In [ ]:
# (例)
# ugroup_name = 'JupyterHub'

ugroup_name = 

### チェック

対象となるVCノードがAnsibleによって操作できることを確認します。

Ansibleの設定ファイルの場所を環境変数に設定しておきます。

In [ ]:
from pathlib import Path
import os

cfg_ansible = Path('ansible.cfg')
if cfg_ansible.exists():
    os.environ['ANSIBLE_CONFIG'] = str(cfg_ansible.resolve())

構築対象となる各VCノードにアクセスできることを確認します。

In [ ]:
target_hub = f'{ugroup_name}_manager'

!ansible {target_hub} -m ping

In [ ]:
target_nodes = f'{ugroup_name}_worker'

!ansible {target_nodes} -m ping

UnitGroup名に対応する group_varsファイルが存在していることを確認します。

In [ ]:
!test -f group_vars/{ugroup_name}

## パラメータの設定

JupyterHubシステムに関する各種設定項目を指定します。

ここで設定された項目はansible変数として保存され、ファイルのmanagerノードへの配置時に各ファイルに反映されます。

### 必須設定項目

#### パラメータ類

In [ ]:
# JupyterHubのホスト名(FQDN) (例) 'www.sample.org'
jupyterhub_fqdn = 

# lti1.3認証連携情報(moodle等)
# lms プラットフォームID (例) 'www.sample.org'
lms_platform_id = 
# lms クライアントID 
lms_cliend_id = 


受講生一覧取得方法を設定します。  
Moodleのバージョンが4.0.0以上であれば、以下のセルを変更する必要はありません。  
Moodleのバージョンが4.0.0未満で、NRPS（Names and Role Provisioning Services）を利用できない場合に、moodleのwebserviceを利用します。（https://tracker.moodle.org/browse/MDL-75279）
その場合、`get_course_member_method`には`'moodle_api'`を指定し、Moodleで発行したトークンを`lms_api_token`に指定してください。

In [ ]:
# get_course_member_method = 'moodle_api'
get_course_member_method = ''
lms_api_token = ''

[configurable-http-proxy](https://github.com/jupyterhub/configurable-http-proxy)で使用するトークンを生成します。

In [ ]:
import random
import string
configproxy_auth_token = ''.join([random.choice("abcdef" + string.digits) for _ in range(32)])
configproxy_auth_token

### 任意設定項目

#### 設定用ファイル配置

テンプレートとして事前準備したファイルを、現在使用している構築用ワークディレクトリにコピーします。

In [ ]:
!mkdir -p edit
!cp -n ./template/jupyterhub/jupyterhub/lms_web_service.py ./edit/
!cp -n ./template/jupyterhub/jupyterhub/jupyterhub_config.py ./edit/
!cp -n ./template/jupyterhub/jupyterhub/jupyterhub_params.yaml ./edit/

#### パラメータ類

jupyterhubで使用するDB等の接続情報等を設定します。

In [ ]:
# メールアドレスドメイン設定
email_domain = 'example.com'

# JupyterhubDB接続情報
db_user = 'jupyter'
db_password = 'PassWordDesu'

# Jupyterhub用LDAPサーバ（ローカルLDAP）
ldap_admin = 'Manager'
ldap_password = 'PassWordDesu'

# Jupyterhub 初期設定
jupyterhub_admin_users = ['admin']

# single-user notebook server コンテナイメージ
singleuser_image = 'mcj-cloudhub-nb:latest'

# 共有ディレクトリパス
home_directory_root = '/jupyter'
share_directory_root = '/exchange'

# dockerネットワーク
swarm_network = 'jupyterhub-network'

# ユーザを一意に識別するキー
lti_username_key = 'sub'

# ユーザのcookieの有効日数(0.25日=6時間)
cookie_max_age_days = 0.25

# Jupyterhubコンテナイメージ
jupyterhub_image = 'mcj-cloudhub:latest'

# single-user notebook server コンテナをDocker Swarmのどのノードで起動するか
# node_role = 'manager'
node_role = 'worker'


single-user notebook serverコンテナのリソース制限等を設定します。  
変更する場合、以下の内容を書き換えてください。  

[参考: DockerSpawner API](https://jupyterhub-dockerspawner.readthedocs.io/en/latest/api/index.html)

また、idle状態のsingle-user notebook serverコンテナを定期的にシャットダウンするサービスを稼働させるため、idle状態でのタイムアウトの時間設定を変更する場合もこちらで行ってください。デフォルトでは、1分に1回、idle状態かどうかのチェックを行い、600秒＝10分間idle状態が続いているコンテナはシャットダウンするように設定しています。  
idle状態のコンテナも起動したままにしたい場合、`cull_server_timeout`の値を0に設定してください。  

[参考: jupyterhub-idle-culler](https://github.com/jupyterhub/jupyterhub-idle-culler)

In [ ]:
import yaml
from pathlib import Path

path = Path('edit/jupyterhub_params.yaml')
with path.open() as f:
    params = yaml.safe_load(f)

params.update({
    'resource': {
        'groups': {
            'student': {
                'mem_limit': '1G',
                'cpu_limit': 0.5,
                'mem_guarantee': 0,
                'cpu_guarantee': 0,
            },
            'teacher': {
                'mem_limit': '1G',
                'cpu_limit': 1.0,
                'mem_guarantee': 0,
                'cpu_guarantee': 0,
            },
        },
    },
    'cull_server': {
        'cull_server_timeout': 600,
        'cull_server_every': 60,
        'cull_server_max_age': 0,
    },
})

with path.open(mode='w') as f:
    yaml.safe_dump(params, stream=f)

#### その他Jupyterhubで設定可能な項目

その他、Jupyterhubで設定可能な項目は、[Jupyterhub公式ドキュメント](https://jupyterhub.readthedocs.io/en/1.4.2/reference/config-reference.html)等を参照し、`jupyterhub_config.py`を直接編集してください。

以下のセルを実行すると、編集対象の`jupyterhub_config.py`の編集画面へのリンクを表示します。

In [ ]:
%run scripts/edit_conf.py
generate_edit_link_config()

### オーバーレイネットワークの指定

JupyterhubHubのコンテナはDocker Swarmのオーバーレイネットワークで実行されます。オーバーレイネットワークに割り当てるサブネットを指定します。

> 基本的にはデフォルト値からの変更は不要ですが、VCノードに割り当てられているIPアドレスと範囲が重複している場合は他のサブネットに変更して下さい。

In [ ]:
# (例)
# jupyterhub_backend = '10.1.0.0/20'

jupyterhub_backend = '10.1.0.0/20'

指定されたサブネットがVCノードに割り当てられているIPアドレスと重なっていないことをチェックします。次のセルを実行してエラーとならないことを確認してください。

In [ ]:
!ansible-playbook -v -e jupyterhub_backend={jupyterhub_backend} -l {ugroup_name} \
    playbooks/check-subnet.yml

### パラメータの保存

ここまで指定したパラメータを ansible の変数として `group_vars`ファイルに保存します。

In [ ]:
import yaml
from pathlib import Path

gvars_path = Path(f'group_vars/{ugroup_name}')
with gvars_path.open() as f:
    gvars = yaml.safe_load(f)

gvars.update({
    'swarm_network': swarm_network,
    'jupyterhub_backend': jupyterhub_backend,
    'jupyterhub_fqdn': jupyterhub_fqdn,
    'db_user': db_user,
    'db_password': db_password,
    'email_domain': email_domain,
    'jupyterhub_admin_users': jupyterhub_admin_users,
    'lms_platform_id': lms_platform_id,
    'lms_cliend_id': lms_cliend_id,
    'lms_api_token': lms_api_token if 'lms_api_token' in locals() else '',
    'get_course_member_method': get_course_member_method if 'get_course_member_method' in locals() else '',
    'singleuser_image': singleuser_image,
    'singleuser_image_no_tag': singleuser_image.split(':')[0],
    'ldap_password': ldap_password,
    'ldap_admin': ldap_admin,
    'home_directory_root': home_directory_root,
    'share_directory_root': share_directory_root,
    'lti_username_key': lti_username_key,
    'cookie_max_age_days': cookie_max_age_days,
    'configproxy_auth_token': configproxy_auth_token,
    'jupyterhub_image': jupyterhub_image,
    'node_role': node_role,
    'ldap_admin': ldap_admin,
})

with gvars_path.open(mode='w') as f:
    yaml.safe_dump(gvars, stream=f)

## JupyterHubのセットアップ

JupyterHubの構成要素となる、各コンテナのセットアップを行います。

### Nginx

Managerノードに構築する、Nginx コンテナに関するセットアップを行います。

#### 証明書の配置

Nginxコンテナで使用するサーバ証明書の配置を確認します。

サーバ証明書を配置するディレクトリを作成します。

In [ ]:
!ansible {target_hub} -b -m file -a \
    'path={{{{base_dir}}}} state=directory owner={{{{ansible_user}}}}'
!ansible {target_hub} -b -m file -a \
    'path={{{{base_dir}}}}/certs state=directory owner={{{{ansible_user}}}}'

作成したディレクトリ`/srv/jupyterhub/certs`に証明書、秘密鍵を配置してください。出どころなどの情報を必要以上に残さないためにNotebookからの操作ではなく、ターミナルなどから **managerノードに ssh でログインして操作を行ってください**。

配置する証明書などのファイル名は以下のようにしてください。

* サーバ証明書と中間CA証明書を連結したもの
    - `/srv/jupyterhub/certs/fullchain.pem`
* 秘密鍵
    - `/srv/jupyterhub/certs/privkey.pem`
    - パスフレーズを無しにする

sshでログインする manager ノードのIPアドレスを確認します。表示されたIPアドレスに対して、ユーザ名`vcp`と「VCノード作成」のNotebookで設定したSSHの秘密鍵を指定することで manager ノードにsshでログインできます。

In [ ]:
!ansible {target_hub} -c local -m debug -a 'var=vc_ipaddress'

#### 証明書の配置確認

証明書の配置後に以下のセルを実行し、正しく配置できていることを確認します。

秘密鍵の内容を表示してみます。

In [ ]:
try:
    !ansible {target_hub} -a \
        'openssl rsa -noout -text -in  {{{{certs_dir}}}}/privkey.pem'
except Exception as e:
    # 鍵の形式が異なる場合、正しく配置できていてもエラーになる場合があります。
    !ansible {target_hub} -a \
         'openssl ec -in {{{{certs_dir}}}}/privkey.pem -text -noout'

中間CA証明書を連結したサーバ証明書の内容を表示してみます。

In [ ]:
!ansible {target_hub} -a \
    'openssl x509 -noout -text -in {{{{certs_dir}}}}/fullchain.pem'

証明書の owner, group などの値を利用環境に合わせて以下のように設定します。

* owner: 33
* group: 33

この設定はコンテナ内では以下のように設定したことに相当します。

* owner: `www-data`
* group: `www-data`

In [ ]:
cert_owner = 33
cert_group = 33
!ansible {target_hub} -b -m file -a \
    'path={{{{certs_dir}}}} owner={cert_owner} \
    group={cert_group} state=directory'
!ansible {target_hub} -b -m file -a \
    'path={{{{certs_dir}}}}/fullchain.pem \
    owner={cert_owner} group={cert_group}'
!ansible {target_hub} -b -m file -a \
    'path={{{{certs_dir}}}}/privkey.pem \
    owner={cert_owner} group={cert_group} mode=0600'

### JupyterHub

JupyterHubコンテナに関するセットアップを行います。

#### JupyterHubコンテナに関するファイルを準備する

JupyterHubコンテナを実行するために必要となるファイルを準備する Ansible Playbook を実行します。

Playbook では以下の処理を行います。

* コンテナイメージのビルド

コンテナイメージのビルドに必要なファイルを配置します。

まず、ファイルを格納するディレクトリを作成

In [ ]:
# 先にディレクトリ作成
!ansible {target_hub} -b -m file -a \
    'path={share_directory_root}/nbgrader state=directory owner={{{{ansible_user}}}}'
!ansible {target_hub} -b -m file -a \
    'path={share_directory_root}/class state=directory owner={{{{ansible_user}}}}'
!ansible {target_hub} -b -m file -a \
    'path={{{{base_dir}}}}/jupyterhub/ldap state=directory owner={{{{ansible_user}}}}'
!ansible {target_hub} -b -m file -a \
    'path={{{{base_dir}}}}/jupyterhub/nginx state=directory owner={{{{ansible_user}}}}'

ファイルの配置をチェックモードで試行

In [ ]:
# 共有ディレクトリに必要なファイル配置
!ansible {target_hub} -CDv -m synchronize \
    -a 'src=template/directories/jupytershare/nbgrader dest={share_directory_root}'
!ansible {target_hub} -CDv -m synchronize \
    -a 'src=template/directories/jupytershare/class dest={share_directory_root}'
!ansible {target_hub} -CDv -m template \
    -a 'src=template/directories/jupytershare/nbgrader/templates/teachers/nbgrader_config.py \
    dest={share_directory_root}/nbgrader/templates/teachers backup=yes'

# ホームディレクトリに必要なファイル配置
# !ansible {target_hub} -CDv -m synchronize \
#     -a 'src=template/directories/skelton dest={home_directory_root}'
!ansible {target_hub} -CDv -m synchronize \
    -a 'src=template/jupyterhub/jupyterhub dest={{{{base_dir}}}}/jupyterhub'
!ansible {target_hub} -CDv -m synchronize \
    -a 'src=template/jupyterhub/nginx dest={{{{base_dir}}}}/jupyterhub'
!ansible {target_hub} -CDv -m synchronize \
    -a 'src=template/jupyterhub/ldap dest={{{{base_dir}}}}/jupyterhub'

# jupyterhubシステム構築に必要なファイル配置
!ansible {target_hub} -CDv -m synchronize \
    -a 'src=template/jupyterhub dest={{{{base_dir}}}}'
!ansible {target_hub} -CDv -m template \
    -a 'src=template/jupyterhub/docker-compose.yml dest={{{{base_dir}}}}/docker-compose.yml backup=yes'
# !ansible {target_hub} -CDv -m synchronize \
#     -a 'src=template/jupyterhub/ldap/ldifs dest={{{{base_dir}}}}/jupyterhub/ldap/ldifs'
!ansible {target_hub} -CDv -m synchronize \
    -a 'src=template/jupyterhub/jupyterhub/sudoers dest={{{{share_directory_root}}}} rsync_opts=--chown=root:root,--chmod=600' --become

実際にファイルを配置

In [ ]:
# 共有ディレクトリに必要なファイル配置
!ansible {target_hub} -Dv -m synchronize \
    -a 'src=template/directories/jupytershare/nbgrader dest={share_directory_root}'
!ansible {target_hub} -Dv -m synchronize \
    -a 'src=template/directories/jupytershare/class dest={share_directory_root}'
!ansible {target_hub} -Dv -m template \
    -a 'src=template/directories/jupytershare/nbgrader/templates/teachers/nbgrader_config.py \
    dest={share_directory_root}/nbgrader/templates/teachers backup=yes'

# ホームディレクトリに必要なファイル配置
!ansible {target_hub} -Dv -m synchronize \
    -a 'src=template/directories/skelton dest={home_directory_root}'

# jupyterhubシステム構築に必要なファイル配置
# !ansible {target_hub} -Dv -m synchronize \
#     -a 'src=template/jupyterhub dest={{{{base_dir}}}}'
!ansible {target_hub} -Dv -m synchronize \
    -a 'src=template/jupyterhub/jupyterhub dest={{{{base_dir}}}}/jupyterhub'
!ansible {target_hub} -Dv -m synchronize \
    -a 'src=template/jupyterhub/nginx dest={{{{base_dir}}}}/jupyterhub'
!ansible {target_hub} -Dv -m synchronize \
    -a 'src=template/jupyterhub/ldap dest={{{{base_dir}}}}/jupyterhub'

!ansible {target_hub} -Dv -m template \
    -a 'src=template/jupyterhub/docker-compose.yml dest={{{{base_dir}}}}/docker-compose.yml backup=yes'

!ansible {target_hub} -Dv -m template \
    -a 'src=template/jupyterhub/nginx/default.conf dest={{{{base_dir}}}}/jupyterhub/nginx/default.conf backup=yes'

!ansible {target_hub} -Dv -m synchronize \
    -a 'src=template/jupyterhub/jupyterhub/sudoers dest={{{{share_directory_root}}}} rsync_opts=--chown=root:root,--chmod=600' --become

#### Jupyterhubイメージのビルド

実際にビルドする前にdryrun（チェックモード）でAnsibleを実行します。

In [ ]:
!ansible-playbook -l {target_hub} -CDv playbooks/setup-jupyterhub.yml || true

実際にコンテナイメージのビルドを行います。

In [ ]:
!ansible-playbook -l {target_hub} playbooks/setup-jupyterhub.yml -e jupyterhub_image={jupyterhub_image}

JupyterHubのコンテナイメージが存在することを確認します。

In [ ]:
!ansible {target_hub} -a \
    'docker images {{{{jupyterhub_image}}}}'

JupyterHubコンテナのために配置したファイルを確認します。

In [ ]:
!ansible {target_hub} -a 'tree {{{{jupyterhub_dir}}}}'

### single-user Jupyter Notebook server

#### イメージが存在しない場合-公開リポジトリからのダウンロード

公開リポジトリからsingle-user Jupyter Notebook serverコンテナイメージをダウンロードします。

以下のセルを実行し、single-user Jupyter Notebook serverコンテナイメージをダウンロードしてください。

ダウンロードが成功した場合、以降のセルは実行せず、「４ コンテナの起動」に進んでください。

In [ ]:
!ansible {ugroup_name} -a 'docker pull {{{{singleuser_image}}}}'

#### イメージが存在しない場合-イメージのビルド

１つ前のセルで、公開リポジトリからのダウンロードが出来ている場合は、ビルドの必要はありません。本セクションはスキップして、「４ コンテナの起動」に進んでください。

`template/notebook` にsingle-user Jupyter Notebook serverコンテナイメージをビルドするためのファイルを格納しています。

オリジナルのイメージを作成する場合は、`template/notebook`の内容を置き換えて以降のセルを実行してください。

まず、イメージのビルドに必要なファイルを格納するためのディレクトリを作成します。

In [ ]:
!ansible {ugroup_name} -b -m file -a \
    'path={{{{notebook_dir}}}} state=directory owner={{{{ansible_user}}}}'

single-user Jupyter Notebook serverコンテナイメージビルドに必要なファイルを配置します。

実際に配置する前に、チェックモードでファイルの不足等が無いか確認します。

In [ ]:
!ansible {ugroup_name} -CDv -m synchronize \
    -a 'src=template/notebook dest={{{{base_dir}}}}'
!ansible {ugroup_name} -CDv -m template \
    -a 'src=template/notebook/image/ldap.conf dest={{{{base_dir}}}}/notebook/image/ backup=yes'
!ansible {ugroup_name} -CDv -m template \
    -a 'src=template/notebook/image/nbgrader_config.py dest={{{{base_dir}}}}/notebook/image/ backup=yes'

ファイルを実際に配置します。

In [ ]:
!ansible {ugroup_name} -Dv -m synchronize \
    -a 'src=template/notebook dest={{{{base_dir}}}}'
!ansible {ugroup_name} -Dv -m template \
    -a 'src=template/notebook/image/ldap.conf dest={{{{base_dir}}}}/notebook/image/ backup=yes'
!ansible {ugroup_name} -Dv -m template \
    -a 'src=template/notebook/image/nbgrader_config.py dest={{{{base_dir}}}}/notebook/image/ backup=yes'

配置したファイルを用いて、single-user Jupyter Notebook serverコンテナイメージのビルドを行います。

ビルド前に、チェックモードで確認します。

In [ ]:
!ansible-playbook -l {ugroup_name} -CDv playbooks/setup-jupyter-notebook.yml || true

実際にビルドを行います。

In [ ]:
!ansible-playbook -l {ugroup_name} playbooks/setup-jupyter-notebook.yml

各VCノードのコンテナイメージ一覧を確認します。

single-user Jupyter Notebook serverコンテナイメージが存在することを確認します。

In [ ]:
print(f'指定したイメージ名:{singleuser_image}')

!ansible {ugroup_name} -m shell -a 'docker images | \
    grep -e "{{{{singleuser_image_no_tag}}}}"'

## コンテナの起動

### docker-compose.yml の配置

複数のコンテナに関する設定をまとめて扱うために `docker-compose.yml` を利用します。

ここまでに指定されたパラメータに応じた`docker-compose.yml`, `default.conf`を生成し、構築環境に配置します。

まずは、チェックモードで確認を行います。

In [ ]:
!ansible {target_hub} -CDv -m template \
    -a 'src=template/jupyterhub/docker-compose.yml dest={{{{base_dir}}}}/jupyterhub backup=yes'
!ansible {target_hub} -CDv -m template \
    -a 'src=template/jupyterhub/nginx/default.conf dest={{{{base_dir}}}}/jupyterhub/nginx backup=yes'
!ansible {target_hub} -CDv -m template \
    -a 'src=edit/lms_web_service.py dest={{{{jupyterhub_dir}}}}/jupyterhub backup=yes'
!ansible {target_hub} -CDv -m template \
    -a 'src=edit/jupyterhub_config.py dest={{{{jupyterhub_dir}}}}/jupyterhub backup=yes'
!ansible {target_hub} -CDv -m template \
    -a 'src=edit/jupyterhub_params.yaml dest={{{{jupyterhub_dir}}}}/jupyterhub backup=yes'

実際に各ファイルの配置を行います。

In [ ]:
!ansible {target_hub} -Dv -m template \
    -a 'src=template/jupyterhub/docker-compose.yml dest={{{{base_dir}}}}/jupyterhub backup=yes'
!ansible {target_hub} -Dv -m template \
    -a 'src=template/jupyterhub/nginx/default.conf dest={{{{base_dir}}}}/jupyterhub/nginx backup=yes'
!ansible {target_hub} -Dv -m template \
    -a 'src=edit/lms_web_service.py dest={{{{jupyterhub_dir}}}}/jupyterhub backup=yes'
!ansible {target_hub} -Dv -m template \
    -a 'src=edit/jupyterhub_config.py dest={{{{jupyterhub_dir}}}}/jupyterhub backup=yes'
!ansible {target_hub} -Dv -m template \
    -a 'src=edit/jupyterhub_params.yaml dest={{{{jupyterhub_dir}}}}/jupyterhub backup=yes'

### コンテナの起動

コンテナを起動します。

In [ ]:
!ansible {target_hub} -a 'chdir={{{{base_dir}}}}/jupyterhub \
    docker stack deploy -c docker-compose.yml {{{{ugroup_name}}}}'

コンテナの起動状態を確認します。

In [ ]:
!ansible {target_hub} -a 'docker stack ps {{{{ugroup_name}}}}'

Masterノードにて、jupyterhubのヘルスチェック用URLに向けて、curlコマンドを実行してみます。    
起動に時間がかかる場合があるため、指定回数を上限とし、成功するまでリトライします。  
ステータスコード`200`が返ると、このセルは正常終了します。  

※managerノード（のVM）へのアクセス制限を行っている場合、この構築環境からはアクセスできない場合があります。その場合は、許可されたアクセス元から、https://{master_fqdn}/hub/healthにGETリクエストを行うか、https://{master_fqdn}/にブラウザでアクセスし、画面が表示されれば問題ありません。
このようなアクセス制限を行う場合は、Jupyterhubに設定しているグローバルIPアドレスからMoodleへのアクセスを許可する必要があります。

In [ ]:
import time

# 規定回数
retry_max = 18
err = None

for retry in range(retry_max):
    try:
        !ansible {target_hub} -m uri -a "url=https://{master_fqdn}/hub/health"
        break

    except Exception as e:
        print("retry")
        err = e
        time.sleep(10)
else:
    raise err

`MariaDB` コンテナのログを表示してみます。

In [ ]:
!ansible {target_hub} -a 'docker service logs {{{{ugroup_name}}}}_mariadb'

`jupyterhub` コンテナのログを表示してみます。

> DBに接続できないなどのエラーが表示されていないことを確認してください。

In [ ]:
!ansible {target_hub} -a 'docker service logs {{{{ugroup_name}}}}_jupyterhub'

`Nginx` コンテナのログを表示してみます。

> 証明書設定に誤りがあるなどのエラーが表示されていないことを確認してください。

In [ ]:
!ansible {target_hub} -a 'docker service logs {{{{ugroup_name}}}}_proxy'

## LMS設定

LTI認証連携で受講生情報を取得するために必要な公開鍵情報を表示します。  
NRPS（Names and Role Provisioning Services）を利用できる場合、表示された公開鍵情報を、事前に設定したMoodleの外部ツール設定にて、RSA鍵の欄にコピー＆ペーストしてください。  
設定例について、「021-Moodleの外部ツール設定例（lti1.3&JupyterHub）」ノートブックの「NRPS利用の場合の設定」の章に図解があります。

In [ ]:
if not get_course_member_method == 'moodle_api':
    !ansible {target_hub} -a \
        'cat  {{{{jupyterhub_dir}}}}/jupyterhub/public_key_nrps.pem'